# Using NER to Tag Data

We are using the dataset pulled from */r/investing* and the SpaCy NER code covered in the previous sections.

In [1]:
import spacy
import pandas as pd

nlp = spacy.load('en_core_web_sm')

The NER code will be reformated into a function that will consume a length of text, and return a list of mentioned organizations:

In [5]:
def get_orgs(text):
    # process the text with our SpaCy model to get named entities
    text = str(text)
    doc = nlp(text)
    # initialize list to store identified organizations
    org_list = []
    # loop through the identified entities and append ORG entities to org_list
    for entity in doc.ents:
        if entity.label_ == 'ORG':
            org_list.append(entity.text)
    # if organization is identified more than once it will appear multiple times in list
    # we use set() to remove duplicates then convert back to list
    org_list = list(set(org_list))
    return org_list

## Applying NER

All we need to do now is load in the */r/investing* data and apply the `get_orgs` function to our text column to create a new `organizations` column.

1. Load the data and view the top five rows with `df.head()`:

In [3]:
df = pd.read_csv('./data/reddit_investing.csv', sep='|')
df.head()

,created_utc,downs,id,score,selftext,subreddit,title,ups,upvote_ratio
0,1.646144e+09,0.0,t3_t48xbu,1.0,My 9yr old wants to invest his 300$ so he can ...,investing,Investment options for children?,1.0,1.00
1,1.646130e+09,0.0,t3_t44pro,597.0,"Again, the world's major chip and semiconducto...",investing,Ukraine supplies 70% of the world's neon. Chip...,597.0,0.95
2,1.646129e+09,0.0,t3_t44j5g,2.0,Have a general question? Want to offer some c...,investing,Daily General Discussion and Advice Thread - M...,2.0,0.76
3,1.646091e+09,0.0,t3_t3t1i6,15.0,Can someone explain how/why something like the...,investing,How would the RSX get Delisted?,15.0,0.77
4,1.646075e+09,0.0,t3_t3n3cz,0.0,Many years ago I had money put in DODBX for me...,investing,trying to figure DODBX out,0.0,0.14


2. Extract mentioned organizations from `selftext` and add to a new column called `organizations`:

In [6]:
df['organizations'] = df['selftext'].apply(get_orgs)
df.head()

,created_utc,downs,id,score,selftext,subreddit,title,ups,upvote_ratio,organizations
0,1.646144e+09,0.0,t3_t48xbu,1.0,My 9yr old wants to invest his 300$ so he can ...,investing,Investment options for children?,1.0,1.00,[]
1,1.646130e+09,0.0,t3_t44pro,597.0,"Again, the world's major chip and semiconducto...",investing,Ukraine supplies 70% of the world's neon. Chip...,597.0,0.95,[]
2,1.646129e+09,0.0,t3_t44j5g,2.0,Have a general question? Want to offer some c...,investing,Daily General Discussion and Advice Thread - M...,2.0,0.76,[]
3,1.646091e+09,0.0,t3_t3t1i6,15.0,Can someone explain how/why something like the...,investing,How would the RSX get Delisted?,15.0,0.77,"[ADR, E-Trade, RSX]"
4,1.646075e+09,0.0,t3_t3n3cz,0.0,Many years ago I had money put in DODBX for me...,investing,trying to figure DODBX out,0.0,0.14,[]


*(This step can take a long time to run. It can be useful to break larger datasets into more manageable chunks if required)*

Now we have a list of all mentioned organizations contained within the `organizations` column. We will now take the full column, merge each list, and use `Counter` to create a frequency table of organization mentions.

In [15]:
# merge organizations column into one big list
orgs = df['organizations'].to_list()
orgs = [org for sublist in orgs for org in sublist]
orgs[:10]

['ADR',
 'E-Trade',
 'RSX',
 'Gydan',
 'The Board of Shell',
 'Nord Stream',
 'https://www.shell.com/media/news-and-media-releases/2022/shell-intends-to-exit-equity-partnerships-held-with-gazprom-entities.html\n&',
 'Shell',
 'the Salym Petroleum Development',
 'ACH']

In [8]:
from collections import Counter

In [16]:
# create dictionary of organization mention frequency
org_freq = Counter(orgs)

We now have a *Counter* dictionary containing all of our organization labels as *keys*, and their mention frequency as *values*. The `most_common(n)` method allows us to view the **n** most frequently mentioned organizations:

In [17]:
org_freq.most_common(10)

[('Fed', 32),
 ('Fidelity', 19),
 ('Amazon', 19),
 ('EPS', 16),
 ('Apple', 15),
 ('SPY', 14),
 ('Microsoft', 14),
 ('Vanguard', 12),
 ('NASDAQ', 11),
 ('EU', 10)]

Clearly there is a need to do some further pruning of the data to remove non-organization labels like *EV* (electric vehicle), *COVID*, *ETF* (exchange trading fund), *NYSE*, and *SPAC* (special acquisition company). Depending on the use-case it may even be useful to keep a few of these, or remove a few others like *SEC* and *FDA*.

To do this, we would create a custom list and implement it in our `get_orgs` function like so:

In [13]:
BLACKLIST = ['ev', 'covid', 'etf', 'nyse', 'sec', 'spac', 'fda']

def get_orgs(text):
    doc = nlp(str(text))
    org_list = []
    for entity in doc.ents:
        # here we modify the original code to check that entity text is not equal to one of our 'blacklisted' organizations
        # (we also add .lower() to lowercase the text, this allows us to match both 'nyse' and 'NYSE' with just 'nyse')
        if entity.label_ == 'ORG' and entity.text.lower() not in BLACKLIST:
            org_list.append(entity.text)
    # if organization is identified more than once it will appear multiple times in list
    # we use set() to remove duplicates then convert back to list
    org_list = list(set(org_list))
    return org_list

If we then apply this new `get_org` function to our dataframe we will see that *ETF* that originally appeared alongside *ARK* in the second row is no longer included:

In [14]:
df['organizations'] = df['selftext'].apply(get_orgs)
df.head()

,created_utc,downs,id,score,selftext,subreddit,title,ups,upvote_ratio,organizations
0,1.646144e+09,0.0,t3_t48xbu,1.0,My 9yr old wants to invest his 300$ so he can ...,investing,Investment options for children?,1.0,1.00,[]
1,1.646130e+09,0.0,t3_t44pro,597.0,"Again, the world's major chip and semiconducto...",investing,Ukraine supplies 70% of the world's neon. Chip...,597.0,0.95,[]
2,1.646129e+09,0.0,t3_t44j5g,2.0,Have a general question? Want to offer some c...,investing,Daily General Discussion and Advice Thread - M...,2.0,0.76,[]
3,1.646091e+09,0.0,t3_t3t1i6,15.0,Can someone explain how/why something like the...,investing,How would the RSX get Delisted?,15.0,0.77,"[ADR, E-Trade, RSX]"
4,1.646075e+09,0.0,t3_t3n3cz,0.0,Many years ago I had money put in DODBX for me...,investing,trying to figure DODBX out,0.0,0.14,[]


We can save our dataframe with the new *organizations* column to file for later use.

In [18]:
df.to_csv('./data/processed/reddit_investing_ner.csv', sep='|', index=False)